In [ ]:
import math
import random

In [ ]:
def distL2(x1,y1, x2,y2):
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    n = len(coord)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()

        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
def custo(tour):
    z = D[tour[-1], tour[0]]
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]]
    return z,

In [ ]:
def randtour(n):
    sol = list(range(n))
    random.shuffle(sol)
    return sol


In [ ]:
dataset = 'a280.tsp'
path = './'+dataset
print(path)

n, coord, D = read_tsplib(path)

./a280.tsp


In [ ]:
print("Solução aletória:")
tour = randtour(n)
z = custo(tour)
print(f'Rota: {tour}\ncusto: {z}')

Solução aletória:
Rota: [183, 39, 245, 58, 16, 174, 17, 270, 108, 213, 170, 119, 73, 219, 199, 211, 264, 96, 134, 207, 224, 52, 45, 138, 267, 2, 274, 122, 151, 238, 226, 214, 204, 248, 109, 153, 244, 36, 55, 97, 253, 68, 148, 77, 18, 28, 42, 19, 249, 75, 48, 175, 152, 240, 186, 123, 154, 273, 24, 222, 190, 101, 120, 251, 252, 146, 3, 33, 201, 265, 113, 276, 50, 188, 1, 261, 189, 6, 173, 78, 41, 268, 205, 106, 74, 20, 257, 198, 72, 79, 38, 194, 202, 53, 131, 187, 40, 82, 114, 29, 150, 217, 200, 127, 149, 116, 243, 93, 195, 49, 121, 25, 117, 256, 31, 231, 155, 179, 132, 141, 65, 110, 227, 88, 124, 206, 10, 266, 8, 43, 158, 260, 104, 125, 255, 172, 239, 278, 94, 126, 233, 221, 279, 62, 112, 208, 135, 143, 232, 178, 46, 218, 145, 275, 99, 272, 89, 144, 166, 84, 139, 103, 176, 247, 9, 242, 234, 92, 236, 160, 107, 11, 22, 159, 66, 4, 34, 163, 137, 57, 91, 216, 246, 229, 184, 169, 128, 35, 47, 129, 171, 111, 182, 157, 102, 95, 71, 181, 140, 223, 63, 210, 133, 54, 254, 23, 196, 136, 59, 12, 56

In [ ]:
!pip install deap

In [ ]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
toolbox = base.Toolbox()

creator.create("Minimizacao", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Minimizacao)

toolbox.register("aleatorios", random.sample, range(n), n)
toolbox.register("gene", tools.initIterate, creator.Individual, toolbox.aleatorios)
toolbox.register("population", tools.initRepeat, list, toolbox.gene)

toolbox.register("evaluate", custo)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selRoulette)


In [ ]:
def statistics(individual):
  return individual.fitness.values

In [ ]:
vezes = 10

for n in range(vezes):

    populacao = toolbox.population(100)

    prob_cross = 0.8
    prob_mut = 0.2
    geracoes = 100


    stats = tools.Statistics(statistics)
    stats.register("média", numpy.mean)
    stats.register("desvio padrão", numpy.std)

    populacao, info = algorithms.eaSimple(
        populacao,
        toolbox,
        prob_cross,
        prob_mut,
        geracoes,
        stats,
        verbose=False
)

    melhores = tools.selBest(populacao, 1)

    for individuo in melhores:
        print(individuo)
        print("Melhor Solução:", individuo.fitness)
        print("--------------------------------------")
        estatisticas = stats.compile(populacao)
        print(estatisticas)

[34, 124, 77, 178, 184, 28, 56, 214, 224, 65, 142, 29, 206, 216, 204, 87, 252, 117, 108, 40, 68, 13, 275, 2, 191, 59, 188, 18, 102, 146, 133, 93, 26, 268, 170, 181, 122, 58, 169, 107, 16, 101, 242, 235, 50, 11, 149, 159, 7, 260, 132, 232, 36, 81, 138, 4, 193, 177, 27, 97, 161, 9, 75, 88, 84, 71, 57, 226, 74, 196, 157, 246, 104, 137, 30, 129, 261, 239, 258, 211, 176, 145, 241, 227, 85, 190, 208, 139, 259, 238, 21, 112, 249, 240, 73, 171, 203, 270, 120, 244, 103, 66, 48, 233, 185, 174, 83, 125, 180, 53, 79, 262, 212, 1, 278, 162, 256, 15, 253, 152, 218, 154, 126, 225, 276, 91, 221, 96, 100, 49, 222, 207, 266, 210, 195, 192, 200, 44, 243, 267, 269, 141, 236, 163, 80, 105, 144, 76, 151, 175, 46, 265, 153, 89, 42, 271, 219, 247, 197, 25, 86, 168, 150, 248, 131, 220, 277, 179, 20, 67, 43, 5, 273, 147, 90, 123, 187, 99, 38, 51, 250, 98, 213, 113, 199, 231, 12, 31, 23, 279, 19, 251, 3, 158, 95, 60, 14, 17, 165, 62, 202, 272, 255, 172, 92, 111, 201, 61, 10, 234, 229, 245, 140, 264, 194, 189, 72

### A seleção SelRoulette apresenta uma solução pior se comparada a seleção SelTournament, isso se deve ao fato da SelRoulette ser melhor usada para maximização, onde individuos com maiores valores são favorecidos. Como a função é de minimização, o SelRoulette não apresenta uma boa saida.